In [13]:
import pandas as pd
"""
Right now we are downloading data from 
https://fragilestatesindex.org/
Then processing it for move up to red
We need to automate this process for use in our new green to red pipeline(WIP)
On a monthly basis we need to do a data pull and get any new data
If we put it in a dash app we can use POSIT CONNECT to keep the script persisting
"""
df = pd.read_excel("Rankings.xlsx")

cols = []
for col in df.columns:
    if "Unnamed" in col:
        cols.append(df[col].iloc[0])
    else:
        cols.append(col)
        
df.columns = cols
df = df[1:]

df = df[["Country", "Total"]]
df.columns = ["Country", "FSI Total"]

df["FSI Total Normalized"] = df["FSI Total"].apply(lambda x: (x-min(df["FSI Total"]))/(max(df["FSI Total"])-min(df["FSI Total"])))
df["FSI Total Normalized x4"] = df["FSI Total Normalized"]*4

In [48]:
df

,Country,FSI Total,FSI Total Normalized,FSI Total Normalized x4
1,Somalia,111.3,1.000000,4.000000
2,Sudan,109.3,0.979716,3.918864
3,South Sudan,109.0,0.976673,3.906694
4,Syria,108.1,0.967546,3.870183
5,Congo Democratic Republic,106.7,0.953347,3.813387
...,...,...,...,...
175,Denmark,15.9,0.032454,0.129817
176,New Zealand,15.9,0.032454,0.129817
177,Iceland,15.2,0.025355,0.101420
178,Finland,14.3,0.016227,0.064909


In [47]:
df.to_excel("FSI_output.xlsx")